Positional Encoding

In [1]:
import numpy as np
import math
def positional_encoding(max_len, d_model):
    # max_len: độ dài tối đa của 1 chuỗi
    # d_model: kích thước embedding
    pe = np.zeros((max_len, d_model))
    for pos in range(max_len):
        for i in range(0, d_model, 2):
            angle = pos / (10000 ** (i / d_model))
            # Vị trí chẵn
            pe[pos, i] = math.sin(angle)
            # Vị trí lẻ
            pe[pos, i+1] = math.cos(angle)
    return pe



In [2]:
max_len = 5
d_model = 4
pe = np.zeros((5, 4))
print(f"ma trận ban đầu:\n{pe}")
print(f"------------------")
for pos in range(max_len):
    for i in range(0, d_model, 2):
        angle = pos / (10000 ** (i / d_model))
        pe[pos][i] = math.sin(angle)
        pe[pos][i+1] = math.cos(angle)
print(f"ma trận positional encoding:\n{pe}")
print(f"--------------------------")
for i in range(max_len):
    print(f"vector PE cho vị trí thứ: {i} là:\n{pe[i]}")
    print(f"------------------------------")


ma trận ban đầu:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
------------------
ma trận positional encoding:
[[ 0.          1.          0.          1.        ]
 [ 0.84147098  0.54030231  0.00999983  0.99995   ]
 [ 0.90929743 -0.41614684  0.01999867  0.99980001]
 [ 0.14112001 -0.9899925   0.0299955   0.99955003]
 [-0.7568025  -0.65364362  0.03998933  0.99920011]]
--------------------------
vector PE cho vị trí thứ: 0 là:
[0. 1. 0. 1.]
------------------------------
vector PE cho vị trí thứ: 1 là:
[0.84147098 0.54030231 0.00999983 0.99995   ]
------------------------------
vector PE cho vị trí thứ: 2 là:
[ 0.90929743 -0.41614684  0.01999867  0.99980001]
------------------------------
vector PE cho vị trí thứ: 3 là:
[ 0.14112001 -0.9899925   0.0299955   0.99955003]
------------------------------
vector PE cho vị trí thứ: 4 là:
[-0.7568025  -0.65364362  0.03998933  0.99920011]
------------------------------


Sau khi tính PE, ta cộng vào embedding ban đầu: X' = X + PE

In [3]:
# Giả sử vector X ban đầu có giá trị:
x = np.random.randn(max_len, d_model)
print(f"vector embedding của input:\n{x}")
x = x + pe
print(f"---------------------------------")
print(f"vector x khi thêm positional encoding:\n{x}")

vector embedding của input:
[[-1.18367899 -1.03117039 -0.12096149 -0.77137454]
 [ 1.37964519  0.11589549  2.72205184  2.23807625]
 [-0.89925132 -0.57266571 -0.76358526 -0.56746333]
 [ 1.08799885 -1.32705247 -1.95006127 -0.77080067]
 [ 0.61300874  1.69734942  0.85602051 -0.04181455]]
---------------------------------
vector x khi thêm positional encoding:
[[-1.18367899 -0.03117039 -0.12096149  0.22862546]
 [ 2.22111617  0.6561978   2.73205167  3.23802625]
 [ 0.0100461  -0.98881254 -0.7435866   0.43233667]
 [ 1.22911886 -2.31704496 -1.92006577  0.22874936]
 [-0.14379376  1.0437058   0.89600985  0.95738556]]


Multihead Attention

Residual and Normalization Layer

In [24]:
import torch
import torch.nn as nn
# Layer normalization
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        # gamma và beta
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps
    def forward(self, x):
        # x shape: (batch_size, seq_len, d_model)
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased= False)

        x_hat = (x - mean) / torch.sqrt(var + self.eps)
        return self.gamma * x_hat + self.beta

In [5]:
d_model = 5
gamma1 = nn.Parameter(torch.zeros(d_model))
gamma2 = torch.zeros(d_model)
print(f"gamma1:\n{gamma1}")
print(f"gamma2:\n{gamma2}")
print(f"---------------------------")
matrix = torch.zeros(3,4)
print(matrix)

gamma1:
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)
gamma2:
tensor([0., 0., 0., 0., 0.])
---------------------------
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [6]:
# Tạo input gồm 2 câu batch_size = 2
batch_size = 2
seq_len = 3
d_model = 5
x = torch.randn(batch_size,seq_len, d_model)
print(x)
mean = x.mean(dim=-1, keepdim=True)
print(f"Trung bình của từng hàng: \n{mean}")
var = x.var(dim = -1, keepdim=True, unbiased=False)
print(f"Phương sai của từng hàng:\n{var}")

tensor([[[-0.5987, -0.2320, -0.6057,  0.9324, -2.0342],
         [-1.7219,  1.8953,  1.2115, -1.0103,  1.3189],
         [ 0.3567,  0.6332,  0.4815,  1.3752, -0.9833]],

        [[ 0.2044, -1.8340,  1.5005, -0.6044, -1.3318],
         [-0.2383, -1.9353, -1.7706,  2.0652,  0.0875],
         [-0.5819,  0.5200,  1.5788, -0.2002, -0.1366]]])
Trung bình của từng hàng: 
tensor([[[-0.5076],
         [ 0.3387],
         [ 0.3726]],

        [[-0.4130],
         [-0.3583],
         [ 0.2360]]])
Phương sai của từng hàng:
tensor([[[0.8996],
         [2.0424],
         [0.5847]],

        [[1.3886],
         [2.1135],
         [0.5764]]])


In [7]:
gamma = torch.randn(d_model)
print(f"gamma:\n{gamma}")
x_hat = (x - mean) / torch.sqrt(var + 1e-6)
print(f"Chuẩn hóa các giá trị của input:\n{x_hat}")
ans = gamma * x_hat + gamma
print(f"Đầu ra layernorm của input:\n{ans}")

gamma:
tensor([-1.6767, -0.4194,  0.7957,  1.1300,  1.5502])
Chuẩn hóa các giá trị của input:
tensor([[[-0.0960,  0.2906, -0.1034,  1.5183, -1.6095],
         [-1.4419,  1.0892,  0.6107, -0.9440,  0.6859],
         [-0.0209,  0.3407,  0.1423,  1.3111, -1.7732]],

        [[ 0.5240, -1.2059,  1.6239, -0.1624, -0.7797],
         [ 0.0826, -1.0847, -0.9715,  1.6670,  0.3066],
         [-1.0774,  0.3741,  1.7687, -0.5746, -0.4908]]])
Đầu ra layernorm của input:
tensor([[[-1.5157, -0.5413,  0.7134,  2.8457, -0.9448],
         [ 0.7409, -0.8762,  1.2817,  0.0633,  2.6135],
         [-1.6417, -0.5623,  0.9089,  2.6116, -1.1986]],

        [[-2.5553,  0.0863,  2.0878,  0.9466,  0.3416],
         [-1.8151,  0.0355,  0.0227,  3.0138,  2.0255],
         [ 0.1298, -0.5763,  2.2030,  0.4808,  0.7894]]])


In [8]:
# Sublayer: Feed forward network: 2 hidden layer
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(d_ff, d_model)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        output = self.linear2(x)
        return output

In [10]:
d_model = 4
d_ff = 6
linear1 = nn.Linear(d_model, d_ff)
print(linear1)

Linear(in_features=4, out_features=6, bias=True)


In [9]:
# Residual connection Layernorm
class SublayerConnection(nn.Module):
    # y = layernorm(x + sublayer(x))
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        self.norm = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x, sublayer):
        output = self.norm(x + self.dropout(sublayer(x)))
        return output

In [11]:
dropout = nn.Dropout(0.3)
print(dropout)

Dropout(p=0.3, inplace=False)


In [25]:
# y = layernorm(x + sublayer(x))
# Giả sử input x là 1 câu có 4 từ, mỗi từ được mã hóa thành vector 6 chiều
# ma trận embedding
x = torch.randn(1, 4, 6)
print(f"input:\n{x}")
sublayer = FeedForward(d_model=6, d_ff=10)
sublayer_connection = SublayerConnection(6)
y = sublayer_connection(x, sublayer)
print(f"output của x khi đi qua residual + layer normalization:\n{y}")

input:
tensor([[[-0.5875, -0.5132,  1.1445,  0.9172, -0.7887, -0.8159],
         [-0.1844,  0.5546,  0.8932,  1.6768,  1.3389, -0.4177],
         [-0.4384, -1.7041,  0.7210,  0.3412, -0.4026,  0.9362],
         [-2.5167,  1.0215, -1.0276,  1.5976,  1.0510, -1.6409]]])
output của x khi đi qua residual + layer normalization:
tensor([[[-0.5622, -0.8673,  1.1928,  1.5466, -1.0148, -0.2952],
         [-0.9922, -0.6015, -0.0564,  1.3266,  1.3530, -1.0295],
         [-0.3850, -1.6488,  0.2184,  0.3589, -0.2455,  1.7019],
         [-1.6012,  0.8407, -0.5790,  1.2251,  0.7230, -0.6086]]],
       grad_fn=<AddBackward0>)
